# Entra Agent ID Weather Agent (LangChain create_agent)

This notebook mirrors the agentic flow from langgraph_test.ipynb (mock weather tool, single-shot invoke, and streaming), while retrieving an Entra Agent ID token for each tool call.

**What you will do:**
- Set up Entra Agent ID (Blueprint + Agent Identity)
- Store the required IDs/secrets in environment variables
- Run the same mock weather agent with Entra Agent ID token retrieval

## Azure / Entra setup (one-time)

The steps below are adapted from the 3P-Agent-ID-Demo lab guide.

### Prerequisites
- Azure CLI
- PowerShell 7.5+
- Microsoft Graph PowerShell SDK: `Install-Module Microsoft.Graph -Scope CurrentUser`
- Entra role: Global Administrator, Cloud Application Administrator, or Application Administrator

### Authenticate
```bash
az login --use-device-code --tenant <your-tenant-id>
```
```powershell
pwsh
$tenantId = (az account show --query tenantId -o tsv)
Connect-MgGraph -Scopes "AgentIdentityBlueprint.AddRemoveCreds.All","AgentIdentityBlueprint.Create","DelegatedPermissionGrant.ReadWrite.All","Application.Read.All","AgentIdentityBlueprintPrincipal.Create","AppRoleAssignment.ReadWrite.All","User.Read" -TenantId $tenantId
Get-MgContext
```

### Create Blueprint + Agent Identity (automated workflow)
```powershell
# From the 3P-Agent-ID-Demo repo root
. ./EntraAgentID-Functions.ps1
$result = Start-EntraAgentIDWorkflow -Permissions @('User.Read.All')

# Capture these values for the notebook env vars:
$result.Connection.TenantId
$result.Blueprint.BlueprintAppId
$result.Blueprint.ClientSecret
$result.Agent.AgentIdentityAppId
```

### Required environment variables
Create a .env file (or set environment variables) with:
```text
ENTRA_TENANT_ID=...
ENTRA_BLUEPRINT_APP_ID=...
ENTRA_BLUEPRINT_CLIENT_SECRET=...
ENTRA_AGENT_APP_ID=...

AZURE_OPENAI_ENDPOINT=...
AZURE_OPENAI_API_VERSION=...
AZURE_OPENAI_DEPLOYMENT=...
```

In [ ]:
# Optional: install dependencies
#%pip install -q langchain-openai langchain-core python-dotenv requests azure-identity

In [ ]:
import asyncio
import os
from random import randint
from typing import Annotated
import requests
from dotenv import load_dotenv
from pydantic import Field

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_agent

In [ ]:
load_dotenv()

REQUIRED_ENV = [
    "ENTRA_TENANT_ID",
    "ENTRA_BLUEPRINT_APP_ID",
    "ENTRA_BLUEPRINT_CLIENT_SECRET",
    "ENTRA_AGENT_APP_ID",
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_DEPLOYMENT",
]

missing = [key for key in REQUIRED_ENV if not os.getenv(key)]
if missing:
    raise EnvironmentError(f"Missing environment variables: {', '.join(missing)}")

TENANT_ID = os.getenv("ENTRA_TENANT_ID")
BLUEPRINT_APP_ID = os.getenv("ENTRA_BLUEPRINT_APP_ID")
BLUEPRINT_CLIENT_SECRET = os.getenv("ENTRA_BLUEPRINT_CLIENT_SECRET")
AGENT_APP_ID = os.getenv("ENTRA_AGENT_APP_ID")

TOKEN_URL = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

In [ ]:
def get_t1_token():
    data = {
        "client_id": BLUEPRINT_APP_ID,
        "client_secret": BLUEPRINT_CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "api://AzureADTokenExchange/.default",
        "fmi_path": AGENT_APP_ID,
    }
    resp = requests.post(TOKEN_URL, data=data)
    resp.raise_for_status()
    return resp.json()["access_token"]

def get_t2_token(t1_token: str):
    data = {
        "client_id": AGENT_APP_ID,
        "grant_type": "client_credentials",
        "scope": "https://graph.microsoft.com/.default",
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": t1_token,
    }
    resp = requests.post(TOKEN_URL, data=data)
    resp.raise_for_status()
    return resp.json()["access_token"]

def get_agent_token():
    t1 = get_t1_token()
    return get_t2_token(t1)

In [ ]:
@tool
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")]
) -> str:
    """Get the weather for a given location (mock)."""
    _ = get_agent_token()  # Entra Agent ID token retrieved for audit/logging patterns
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."


In [ ]:
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default",
)

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=token_provider,
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    temperature=0,
 )

agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful weather agent.",
)

In [ ]:
query = "What's the weather like in Seattle?"
result = agent.invoke({"messages": [("user", query)]})
result["messages"][-1].content


In [ ]:
async def stream_agent(query: str):
    print("Agent:", end=" ", flush=True)
    async for event in agent.astream_events(
        {"messages": [("user", query)]},
        version="v2",
    ):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if getattr(chunk, "content", None):
                print(chunk.content, end="", flush=True)
    print()

await stream_agent("What's the weather like in Portland?")
